<a href="https://colab.research.google.com/github/srivathsak/Paraphrasing_tool/blob/main/Paraphrasing_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=830bb6555a18b7653f3da605086c3147dcacb2f3f870e6235ff0b8713bd573a8
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [2]:
pip install sentence-splitter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 2.7 MB/s 


In [3]:
pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 34.3 MB/s 


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 29.1 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 50.9 MB/s 
     |████████████████████████████████| 596 kB 52.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.8 MB 20.7 MB/s 


In [6]:
pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
pip install flask_wtf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 40.7 MB/s 


In [8]:
pip install fpdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40725 sha256=9c0207a265d4017c82892e73bd1cc0dcf8db490214f316bf01598ee31e2eb4c2
  Stored in directory: /root/.cache/pip/wheels/d7/ca/c8/86467e7957bbbcbdf4cf4870fc7dc95e9a16404b2e3c3a98c3
Successfully built fpdf


In [9]:
cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [10]:
!ngrok authtoken 4hEnikYGKvwXRRDwWkyYK_wXwdsoDszDKbBU2JBZrq 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [11]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from flask import Flask, flash, request, render_template, send_file
from flask_wtf import FlaskForm
from wtforms import FileField, SubmitField
from werkzeug.utils import secure_filename
import fitz
import time
import textwrap
from fpdf import FPDF
from pathlib import Path
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import warnings
warnings.filterwarnings("ignore")
import os
from wtforms.validators import InputRequired
from flask_ngrok import run_with_ngrok
import datetime

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

def paraphrase_it(context):
  splitter = SentenceSplitter(language='en')
  sentence_list = splitter.split(context)
  paraphrase = []
  for i in sentence_list:
    a = get_response(i,1)
    paraphrase.append(a)
  paraphrase2 = [' '.join(x) for x in paraphrase]
   # Combines the above list into a paragraph
  paraphrase3 = [' '.join(x for x in paraphrase2) ]
  paraphrased_text = str(paraphrase3).strip('[]').strip("'")
  paraphrased_text1= paraphrased_text.replace('"', '')
  return paraphrased_text1




app = Flask(__name__)
run_with_ngrok(app)
app.config['SECRET_KEY'] = 'supersecretkey'
app.config['UPLOAD_FOLDER'] = 'static/files'

class UploadFileForm(FlaskForm):
    file = FileField("File", validators=[InputRequired()])
    submit = SubmitField("UPLOAD")


var_list=[]

def text_to_pdf(text, filename):
    a4_width_mm = 210
    pt_to_mm = 0.35
    fontsize_pt = 10
    fontsize_mm = fontsize_pt * pt_to_mm
    margin_bottom_mm = 10
    character_width_mm = 7 * pt_to_mm
    width_text = a4_width_mm / character_width_mm

    pdf = FPDF(orientation='P', unit='mm', format='A4')
    pdf.set_auto_page_break(True, margin=margin_bottom_mm)
    pdf.add_page()
    pdf.set_font(family='Courier', size=fontsize_pt)
    splitted = text.split('\n')

    for line in splitted:
        lines = textwrap.wrap(line, width_text)

        if len(lines) == 0:
            pdf.ln()

        for wrap in lines:
            pdf.cell(0, fontsize_mm, wrap, ln=1)

    pdf.output(filename, 'F')

@app.route('/download', methods=['GET',"POST"])
def submit1():
  form=UploadFileForm()
  if var_list!=[]:
    filename= var_list.pop()
    if filename.endswith('.pdf'):
      with fitz.open(filename) as doc:
          text=""
          for page in doc:
            text+=page.get_text().replace('\n','')
    elif filename.endswith('.txt'):
      with open(filename, 'r') as file:
        text = file.read().replace('\n', '')
    context=text
    print(context)
    paraphrased_text=paraphrase_it(context)
    print(paraphrased_text)
    os.remove(filename)
    downloadvalue=request.form['fileext']
    if downloadvalue=='PDF':
      suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S%f")
      output_filename="paraphrased_text"+suffix+".pdf"
      text_to_pdf(paraphrased_text, output_filename)
      #print(output_filename+" has been successfully saved in /content/drive/MyDrive/Colab Notebooks/"+output_filename)
      return send_file(output_filename,as_attachment=True)
    elif downloadvalue=='TXT':
      suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S%f")
      output_filename="paraphrased_text"+suffix+".txt"
      path=os.getcwd()
      fname=path+"/"+output_filename
      Path(fname).touch()
      with open(fname, 'w') as f:
        f.write(paraphrased_text)
        f.close()
        print(fname+" has been successfully saved ")
        return send_file(fname,as_attachment=True)
    else:
      pass
  else:
    errormsg="You did not Upload any File!!!"
  # removingfile=output_files.pop()
  # os.remove(removingfile)
  return render_template('index1.html',form=form,error_msg=errormsg)

@app.route('/submit', methods=['GET',"POST"])
def submit():
  form=UploadFileForm()
  text=request.form['text']
  context=text
  print(context)
  paraphrased_text=paraphrase_it(context)
  print(paraphrased_text)
  return render_template('index.html',paraphrased_text=paraphrased_text,original_text=text,form=form)


@app.route('/upload',methods=['GET','POST'])
def upload_file():
  error_msg=""
  form = UploadFileForm()
  if form.validate_on_submit():
      file = form.file.data # First grab the file
      file1=file.filename
      suffix = datetime.datetime.now().strftime("%y%m%d_%H%M%S%f")
      if file1.endswith('.pdf'):
        oldfile=file1
        oldfile1=oldfile.split(".")
        newfile="_".join([oldfile1[0], suffix])
        newfile1=".".join([newfile, 'pdf'])
        file.save(os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(newfile1)))
        flash('File Uploaded!') 
        var_list.append(os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(newfile1)))
      elif file1.endswith('.txt'):
        oldfile=file1
        oldfile1=oldfile.split(".")
        newfile="_".join([oldfile1[0],suffix])
        newfile1=".".join([newfile,'txt'])
        file.save(os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(newfile1))) 
        flash('File Uploaded!')
        var_list.append(os.path.join(app.config['UPLOAD_FOLDER'],secure_filename(newfile1)))
      else:
        error_msg="File Type Not Supported"
  return render_template('index1.html', form=form,error_msg=error_msg)

@app.route('/', methods=['GET',"POST"])
@app.route('/home', methods=['GET',"POST"])
def home():
    var_list=[]
    downloadvalue=""
    form=UploadFileForm()
    return render_template("index.html",form=form)




if __name__ == '__main__':
    app.run()

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

...

 * Running on http://9ea7-34-143-216-221.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [31/May/2022 09:16:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 09:16:43] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/May/2022 09:16:56] "GET /upload HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2022 09:17:00] "GET /home HTTP/1.1" 200 -


hello how are you. What are you doing?


127.0.0.1 - - [31/May/2022 09:17:44] "POST /submit HTTP/1.1" 200 -


How are you? What are you doing?


In [ ]:
os.getcwd()

'/content/drive/MyDrive/Colab Notebooks'